In [ ]:
# Load necessary libraries
library(dplyr)
library(Matrix)
library(data.table)
library(Seurat)
library(ggplot2)
library(RColorBrewer)
library(cowplot)
library(tidyverse)
library(tidyr)
library(viridis)
library(harmony)
library(MASS) #
library(scales)
library(AnnotationHub)	
library(org.Hs.eg.db)  
library(clusterProfiler)
library(biomaRt)
library(ggthemes)
library(msigdbr)
library(conflicted)
library(stringr)


In [ ]:
##

# Get the expression matrix
ExprMat = GetAssayData(seuobj, slot = 'data', assay = 'RNA')
half <- round(ncol(ExprMat) / 2)
n1 <- round(half / 2)
n2 <- round(ncol(ExprMat) * 0.75)

# Split the expression matrix

ExprMat_1 <- ExprMat[, 1:n1]
ExprMat_2 <- ExprMat[, (n1 + 1):half]
ExprMat_3 <- ExprMat[, (half + 1):n2]
ExprMat_4 <- ExprMat[, (n2 + 1):ncol(ExprMat)]

# Compute cell rankings
cells_rankings_list <- lapply(list(ExprMat_1, ExprMat_2, ExprMat_3, ExprMat_4), AUCell_buildRankings)
cells_rankings <- do.call(cbind, cells_rankings_list)

# Compute AUC values
AUC.list <- lapply(1:length(geneset.use$Description), function(i) {
  gene.list <- str_split(geneset.use[i, "geneID"], pattern = '/') %>% unlist() %>% toupper() %>% unique()
  cells_AUC <- AUCell_calcAUC(gene.list, cells_rankings, nCores = 16)
  getAUC(cells_AUC)
})

# Combine AUC data
AUC.data <- do.call(rbind, AUC.list)
rownames(AUC.data) <- geneset.use$Description

# Save AUC data
save(AUC.data, file = "batch234.FAS_AUC_pathways_0402.data.RData")

# Get metadata
meta <- seuobj@meta.data
area <- meta$CellType_batch4_m[match(colnames(AUC.data), rownames(meta))]
sample <- meta$Class[match(colnames(AUC.data), rownames(meta))]
info.data <- data.frame(area = area, sample = sample)

# Calculate expression levels for each group
result.list <- lapply(1:nrow(AUC.data), function(i) {
  info.data$expr <- AUC.data[i,]
  result <- aggregate(expr ~ sample + area, data = info.data, FUN = mean)
  result$term <- rownames(AUC.data)[i]
  result
})

# Combine results
result <- do.call(rbind, result.list)
result$group <- paste0(result$sample, "_", result$area)
data_wide <- pivot_wider(result, id_cols = term, names_from = group, values_from = expr)

# Organize data
rownames(data_wide) <- data_wide$term
data_wide <- data_wide[,-1]

# Prepare data for plotting
plot.data.ad <- data_wide[,(1:7)*2-1]
plot.data.con <- data_wide[,(1:7)*2]
plot.data.dis <- data.frame(matrix(NA, nrow = nrow(plot.data.con), ncol = 7))
colnames(plot.data.dis) <- names(table(result$area))

for (i in 1:nrow(plot.data.con)) {
  plot.data.dis[i,] <- plot.data.ad[i,] - plot.data.con[i,]
  rownames(plot.data.dis)[i] <- rownames(plot.data.ad)[i]
}
